In [ ]:
import pandas as pd
import numpy as np
import json
import csv

In [ ]:
#1.确定这几个数据都是干什么的
swiping_card_dataframe = pd.read_csv('./bus_swiping_card_info1.csv')
card_info_dataframe = pd.read_csv('./card_info.csv')
bus_swip_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='刷卡数据',header=0,engine='openpyxl')
bus_station_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='站点数据',header=0,engine='openpyxl')

In [ ]:
#swiping_card_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间
#card_info_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间也是上面的信息，但是时间是去年的。
#用去年的时间的同比数据，去预测今年的同比数据，看是否一致，如果一致，说明经济发展没有变化，如果变大，说明经济好转
#bus_swip_dataframe ：一天的统计信息：各条线路、各个站点的刷卡数量信息
#bus_station_dataframe ： 站点的统计信息：各个站点的经纬度

In [ ]:
line_station_number_sequen_dataframe = \
bus_station_dataframe.groupby(['line_identity','station_number']).agg({'station_name':list,'station_identity':list}).reset_index()
line_station_number_sequen_dataframe['length'] = line_station_number_sequen_dataframe['station_identity'].apply(lambda x:len(x))
line_station_number_sequen_dataframe

In [ ]:
line_station_number_sequen_dataframe['id_station'] = line_station_number_sequen_dataframe.apply(lambda x:list(zip(x['station_identity'],x['station_name'])),axis=1)

In [ ]:
line_station_number_sequen_dataframe

In [ ]:
#获取每条线路上乘客上车站所对应的可能的下车站：估计数据量为：
#构建一个字典：
#road_structure = {line_id:{'stations_nums':int,'stations':{station_id:station_name},'all_stations':[]}}
road_line_station_structure_setting={}
for row_index in range(len(line_station_number_sequen_dataframe)):
    line_id = line_station_number_sequen_dataframe[row_index:row_index+1]['line_identity'].values[0]
    stations_nums = line_station_number_sequen_dataframe[row_index:row_index+1]['station_number'].values[0]
    all_stations = line_station_number_sequen_dataframe[row_index:row_index+1]['station_name'].values[0]
    stations={}
    for instance in line_station_number_sequen_dataframe[row_index:row_index+1]['id_station'].values[0]:
        instance_id = instance[0]
        instance_name = instance[1]
        stations[instance_id]=instance_name
    road_line = {'stations_nums':stations_nums,'stations':stations,'all_stations':all_stations}
    road_line_station_structure_setting[line_id] = road_line
road_line_station_structure_setting

In [ ]:
station_distance_rank_index_list = np.loadtxt('./station_pair_distance_rank_index.csv',delimiter=',',dtype=int)
# 从文件中读取字典
with open('./id_2_station.json', 'r') as file:
    id_2_station = json.load(file)
# 将字典保存到文件中
with open('./station_2_id.json', 'r') as file:
    station_2_id = json.load(file)

In [ ]:
#从card_info_dataframe中找出所有用户一天内的od对信息
card_info_dataframe['custom_day'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[0])
card_info_dataframe['custom_precise_time'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[1])
#sum(card_info_dataframe['partitionday']==card_info_dataframe['custom_day'])
#card_info_dataframe
card_info_dataframe.drop(labels=['custom_time','card_type','consume','data_src','data_load_time','partitionday'],axis=1,inplace=True)

In [ ]:
day_0929_dataframe = card_info_dataframe[card_info_dataframe['custom_day']=='2023-09-29']
day_0929_dataframe

In [ ]:
#按照刷卡id进行聚合
every_user_all_trips_dataframe = \
day_0929_dataframe.groupby('card_identity').agg({'custom_precise_time':list, \
                                                 'line_identity':list, \
                                                 'station_identity':list, \
                                                'car_identity':list},axis=1).reset_index()
#每天承载18万人出行181190人
#乘坐两程以上的有109927人
#乘坐单程的极有可能是流动，也有可能是别的交通方式回。从前后相依有下一段旅程的来确定od对，对于没有后程的暂时不确定。
every_user_all_trips_dataframe['trip_length'] = every_user_all_trips_dataframe['line_identity'].apply(lambda x:len(x))
#every_user_all_trips_dataframe

In [ ]:
trip_great_than_2=every_user_all_trips_dataframe[every_user_all_trips_dataframe['trip_length']!=1].reset_index(drop=True)
#trip_great_than_2
trip_great_than_2['trip'] = trip_great_than_2.apply(lambda x:list(zip(x['custom_precise_time'],x['line_identity'],x['station_identity'])),axis=1)
trip_great_than_2

In [ ]:
can_used_od_trips=[]
def find_leave_station(sorted_index_array:np.ndarray,specified_indices:list):
    # 找到指定索引组中在排序数组中的位置
    positions = [sorted_index_array.index(idx) for idx in specified_indices]
    return sorted_index_array[min(positions)]
for row_index in range(len(trip_great_than_2)):
    trip_list = trip_great_than_2[row_index:row_index+1]['trip'].values[0]
    sorted_trips_list = sorted(trip_list,key=lambda x:x[0],reverse=False)
    for trip_index in range(len(sorted_trips_list)-1):
        this_time,this_line,this_station_id = sorted_trips_list[trip_index]
        next_time,next_line,next_station_id = sorted_trips_list[trip_index+1]
        try:
            this_line_station_name = road_line_station_structure_setting[this_line]['stations'][this_station_id]
            this_line_station_id = station_2_id[this_line_station_name]
        
            all_possible_leave_stations = road_line_station_structure_setting[this_line]['all_stations']
            all_possible_leave_stations_index = [station_2_id[station] for station in all_possible_leave_stations]
        
            next_line_station_name = road_line_station_structure_setting[next_line]['stations'][next_station_id]
            next_line_station_id = station_2_id[next_line_station_name]
            distance_2_next_line_station_ndarray  = station_distance_rank_index_list[next_line_station_id]
            leave_station = find_leave_station(distance_2_next_line_station_ndarray.tolist(),all_possible_leave_stations_index)
            od_trip = (this_line_station_id,this_time,this_line,next_line_station_id)
            can_used_od_trips.append(od_trip)
        except:
            continue

In [ ]:
can_used_od_trips[0:5]
print(id_2_station['795'],id_2_station['1416'])
print(road_line_station_structure_setting[7]['all_stations'])
print(bus_station_dataframe[bus_station_dataframe['line_identity']==7]['station_name'])

In [ ]:
#将可用的od对转化为dataframe：这样就可以使用dataframe进行聚类
#can_used_od_trips[0:5]
can_used_od_trips_dataframe = pd.DataFrame(columns=['start_station_id','time','line','end_station_id'])
for trip_index in range(len(can_used_od_trips)):
    start_station_id,time,line,end_station_id=can_used_od_trips[trip_index]
    can_used_od_trips_dataframe.loc[len(can_used_od_trips_dataframe)] = \
    {'start_station_id':start_station_id, \
                                         'time':time, \
                                         'line':line, \
                                         'end_station_id':end_station_id}

In [ ]:
len(can_used_od_trips)
#一共有169099可以确定的od对数据

In [ ]:
can_used_od_trips_dataframe['hour']= \
can_used_od_trips_dataframe['time'].apply(lambda x:x.split(':')[0])
can_used_od_trips_dataframe['minu']= \
can_used_od_trips_dataframe['time'].apply(lambda x:x.split(':')[1])

In [ ]:
can_used_od_trips_dataframe

In [ ]:
can_used_od_trips_dataframe.to_json('./can_used_od_trips_dataframe.json', orient='records')